In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

#import tensorflow
import tensorflow as tf
import tensorflow_datasets as tfds
tf.logging.set_verbosity(tf.logging.ERROR)

#import other helper libraries
import math
import numpy as np
import matplotlib.pyplot as plt

#improve progress bar display
import tqdm
import tqdm.auto
tqdm.tqdm=tqdm.auto.tqdm

print(tf.__version__) #tf version
tf.enable_eager_execution()

/home/emuca002/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

1.13.1


In [2]:
#import Fashion MNIST dataset
dataset, metadata= tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset=dataset['train'], dataset['test']

In [3]:
#store images as class names
class_names=['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot']

In [4]:
#Data Exploration

#get number of images in train and test dataset
num_train_examples= metadata.splits['train'].num_examples
num_test_examples=metadata.splits['test'].num_examples


In [5]:
#Data preprocessing

#normalize the pixels of each image to a range [0,1]
def normalize(images, labels):
    images=tf.cast(images, tf.float32)
    images /=255
    return images, labels

    #normalize train and test datasets
    train_dataset= train_dataset.map(normalize)
    test_dataset=test_dataset.map(normalize)

In [6]:
#Building the model

model=tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
                          input_shape=(28,28,1)),
    #first layer is Conv2D filters (3,3) retaining the original image size by using padding, and creating 32 output (convoluted) images.
    tf.keras.layers.MaxPooling2D((2,2), strides=2),
    #the 32 outputs are reduced in size using MaxPooling2D (2,2) with a stride of 2
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.relu),
    #this conv2d has a (3,3) kernel, that takes the 32 images as input and outputs 64 images
    tf.keras.layers.MaxPooling2D((2,2), strides=2),
    #these are reduced in size using maxpooling2d (2,2) with a stride of 2
    
    #the next layers are similar to the dense NN done before
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu), #128 neuron followed by 10 node softmax layer
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [7]:
#Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [9]:
#Training the model
BATCH_SIZE=32
train_dataset=train_dataset.repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset=test_dataset.batch(BATCH_SIZE)

model.fit(train_dataset, epochs=2, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

Epoch 1/2


ValueError: Error when checking input: expected conv2d_input to have 4 dimensions, but got array with shape (32, 32, 28, 28, 1)

In [ ]:
#Evaluate accuracy
test_loss, test_accuracy=model.evaluate(test_dataset, steps=math.ceil(num_test_examples/BATCH_SIZE))
print('Accuracy on test dataset: ', test_accuracy)

In [ ]:
#Make Predictions
for test_images, test_labels in test_dataset.take(1):
    test_images=test_images.numpy()
    test_labels=test_labels.numpy()
    predictions=model.predict(test_images)

predictions.shape
predictions[0]
ng.argmax(predictions[0])
test_labels[0]
